<h1> In this notebook, we will implement a neural network that regresses volume against segmentation + depth map </h1>

In [79]:
%load_ext autoreload
%autoreload 2

import glob
import os
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras import backend as K
from matplotlib import pyplot as plt

%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"]="3"
np.random.seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
data_path_base = '/root/data/blender_v3'
depth_map_dir = '{}/{}'.format(data_path_base, 'depth_map')
segmentation_dir = '{}/{}'.format(data_path_base, 'mask')
annotation_dir = '{}/{}'.format(data_path_base, 'annotations')


In [99]:
# Get data ready
number_key = lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[-1])
side_key = lambda x: os.path.splitext(os.path.basename(x))[0].split('_')[0]
all_segmentation_paths = sorted(glob.glob('{}/*.npy'.format(segmentation_dir)), key=number_key)
segmentation_paths = [p for p in all_segmentation_paths if side_key(p) == 'left'] # we are working with left frame only
depth_map_paths = sorted(glob.glob('{}/*.npy'.format(depth_map_dir)), key=number_key)
data_dict = dict(zip(map(number_key, segmentation_paths), zip(segmentation_paths, depth_map_paths)))

# get labels
annotation_paths = sort(glob.glob('{}/*npy'))







/root/data/blender_v3/mask/left_2.npy
/root/data/blender_v3/depth_map/depth_map_2.npy


In [ ]:
def myGenerator(file_paths, steps_per_epoch, BATCH_SIZE, INPUT_SHAPE):
    i = 0
    while True:
        x_batch = np.empty((BATCH_SIZE, INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2]))
        y_batch = np.empty((BATCH_SIZE, INPUT_SHAPE[0], INPUT_SHAPE[1], 1))
        for (ind, j) in enumerate(range(i*BATCH_SIZE, (i+1)*BATCH_SIZE)):
            # pick a random image
            f = np.random.choice(file_paths)
            random_x = np.random.randint(0, 1500-img_size)
            random_y = np.random.randint(0, 1500-img_size)
            xb = np.array(Image.open(f))[random_x:random_x+img_size, random_y:random_y+img_size, :]
            ftruth = f.replace('images', 'labels')
            ftruth = ftruth[:-1]
            yb = np.expand_dims(np.array(Image.open(ftruth))[random_x:random_x+img_size, random_y:random_y+img_size, 0], axis=2)
            yb[yb==255]=1
            if np.random.random() < 0.5:
                xb = flip_axis(xb, 1)
                yb = flip_axis(yb, 1)
            if np.random.random() < 0.5:
                xb = flip_axis(xb, 0)
                yb = flip_axis(yb, 0)
            if np.random.random() < 0.5:
                xb = xb.swapaxes(1, 0)
                yb = yb.swapaxes(1, 0)
            x_batch[ind,...] = xb
            y_batch[ind,...] = yb
        # bunch of augmentation

        x_batch = seq.augment_images(x_batch)
        i += 1
        if i >= steps_per_epoch:
            i = 0
        yield x_batch, y_batch

In [80]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1, n_classes=10, shuffle=True):
        # Initialization
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            segmentation_path, depth_map_path = data_dict[ID]
            X[i,] = np.dstack([np.load(segmentation_path), np.load(depth_map_path)])

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

SyntaxError: invalid syntax (<ipython-input-80-193799cf1049>, line 16)

In [109]:
np.dstack([np.load(segmentation_paths[0]).T, np.load(depth_map_paths[0])])

array([[[ 0.        , 15.56217384],
        [ 0.        , 15.55384636],
        [ 0.        , 15.54553795],
        ...,
        [ 0.        ,  7.60877419],
        [ 0.        ,  7.59293795],
        [ 0.        ,  7.57719374]],

       [[ 0.        , 15.57263088],
        [ 0.        , 15.56428337],
        [ 0.        , 15.55595398],
        ...,
        [ 0.        ,  7.60196686],
        [ 0.        ,  7.58615208],
        [ 0.        ,  7.57042789]],

       [[ 0.        , 15.58313084],
        [ 0.        , 15.57476234],
        [ 0.        , 15.56641293],
        ...,
        [ 0.        ,  7.59516335],
        [ 0.        ,  7.5793705 ],
        [ 0.        ,  7.56366825]],

       ...,

       [[ 0.        , 13.88478756],
        [ 0.        , 13.87733078],
        [ 0.        , 13.86989117],
        ...,
        [ 0.        ,  7.5951705 ],
        [ 0.        ,  7.57937717],
        [ 0.        ,  7.56367493]],

       [[ 0.        , 13.87543201],
        [ 0.        , 13.86